In [9]:
pip install streamlit



  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/e9/07/63a6e890c9b998a6318b46c2a34377fd1a3e01a94c427d82bfb2472b7c16/streamlit-1.30.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for click<9,>=7.0 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-metadata<8,>=1.4 from https://files.pythonhosted.org/packages/c0/8b/d8427f023c081a8303e6ac7209c16e6878f2765d5b59667f3903fbcfd

In [24]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load your preprocessed data
parking_data = pd.read_csv("preprocessed_data.csv")
target_column = 'TOTALSPACECOUNT'
X = parking_data.drop(columns=[target_column, 'NAME', 'ADDRESS'])
y = parking_data[target_column]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and fit the preprocessor
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(exclude=['number']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

preprocessor.fit(X_train)  # Fit the preprocessor on the training data

# Build the linear regression pipeline
linear_regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the linear regression model
linear_regression_pipeline.fit(X_train, y_train)

# Get feature names after one-hot encoding
feature_names = preprocessor.named_transformers_['categorical']['encoder'].get_feature_names_out(input_features=categorical_features)

# Combine numeric and categorical feature names
all_feature_names = np.concatenate([numeric_features, feature_names])

# Extract coefficients from the linear regression model
coefficients = linear_regression_pipeline.named_steps['regressor'].coef_

# Streamlit App
st.title("Linear Regression Model App")

# Add widgets for user input
feature_value = st.slider("Select Feature Value", min_value=0.0, max_value=100.0, value=50.0)

if st.button("Predict"):
    # Make predictions using the selected feature value
    predicted_value = linear_regression_pipeline.predict([[feature_value]])
    st.write(f"Predicted Total Space Count: {predicted_value[0]}")

# Display model details or other relevant information
st.write("Linear Regression Model Details:")
st.write(linear_regression_pipeline.named_steps['regressor'])

# Display feature importance
st.write("Linear Regression Model Feature Importance:")
feature_importance_df = pd.DataFrame({'Feature': all_feature_names, 'Coefficient': coefficients})
st.write(feature_importance_df)

# Additional visualizations or information can be added as needed
